In [64]:
import os
import json
import sqlite3
import pandas as pd
import csv
import plotly.express as px

In [65]:
conn = sqlite3.connect('crimes.db')
c=conn.cursor()
conn.close()

In [66]:
db = sqlite3.connect('crimes.db')
cur = db.cursor()

df = pd.read_sql("""SELECT * 
                FROM crimes
                WHERE "LSOA_name" LIKE 'Barnet%' AND "Crime_type" LIKE 'Burglary'
                """, con=db)
df

,Crime_ID,Month,Reported_by,Falls_within,Longitude,Latitude,Location,LSOA_code,LSOA_name,Crime_type,Last_outcome_category,Context
0,,2010-12,Metropolitan Police Service,Metropolitan Police Service,-0.201877,51.655538,On or near High Street,E01000248,Barnet 001A,Burglary,,
1,,2010-12,Metropolitan Police Service,Metropolitan Police Service,-0.207853,51.654317,On or near The Avenue,E01000248,Barnet 001A,Burglary,,
2,,2010-12,Metropolitan Police Service,Metropolitan Police Service,-0.202510,51.656348,On or near Bruce Road,E01000248,Barnet 001A,Burglary,,
3,,2010-12,Metropolitan Police Service,Metropolitan Police Service,-0.206779,51.654768,On or near The Drive,E01000248,Barnet 001A,Burglary,,
4,,2010-12,Metropolitan Police Service,Metropolitan Police Service,-0.209537,51.655223,On or near Marriott Road,E01000249,Barnet 001B,Burglary,,
...,...,...,...,...,...,...,...,...,...,...,...,...
44187,fda0870be993d2c9352ae14a540bf529c15a7aa9d92a4e...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,
44188,c6c2d9a772695865f4ca33fce59c26c48fe3a1cdac8c22...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.202163,51.559100,On or near Hocroft Road,E01000139,Barnet 041B,Burglary,Under investigation,
44189,e5bab2bd4f31de44515dbf125b85284392d0349139a1d3...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.197843,51.561093,On or near Church Walk,E01000140,Barnet 041C,Burglary,Under investigation,
44190,823465427f8764e250a48312715a4bd87e8012412046c6...,2023-03,Metropolitan Police Service,Metropolitan Police Service,-0.199449,51.563896,On or near Llanelly Road,E01000140,Barnet 041C,Burglary,Under investigation,


 # Graph for crime over the years

In [67]:
import plotly.express as px

grouped = df.groupby('Month').size().reset_index(name='Count')

fig = px.line(grouped, x='Month', y='Count')
fig.update_layout(xaxis_title="Month", yaxis_title="Count")
fig.show()

From this it is pretty apparent that overall there has been a decline in burglaries over the year, but especially  in the beginning of 2020 when lockdown measures for the corona virus started, there was a steep drop.

## Crimes per month

In [68]:
import calendar

df['Year'] = df['Month'].str[:4]
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df = df.sort_values('Month')
df['Month'] = df['Month'].dt.month
df['Month'] = df['Month'].apply(lambda x: calendar.month_name[x])
grouped2 = df.groupby(['Year', 'Month']).size().reset_index(name='Count')

# create a bar graph with separate boxes for each year
fig = px.bar(grouped2, x='Month', y='Count', color='Year')
fig.update_layout(xaxis_title="Month", yaxis_title="Crime Count")
fig.update_xaxes(categoryorder='array', categoryarray= ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
# show the plot
fig.show()

There seems to be less crimes during spring and summer, this could be caused by the fact that there are less hours of light during the summer, but this is of course speculation.
The spike of burglaries in november is also interesting and could be worth looking into.

In [69]:
# create a bar graph with separate boxes for each year
fig = px.bar(grouped2, x='Month', y='Count', color='Year')
fig.update_layout(xaxis_title="Month", yaxis_title="Crime Count", barmode='group')
fig.update_xaxes(categoryorder='array', categoryarray= ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
# show the plot
fig.show()

Using this grouped bar char we can see the downward trend even better, the grouped bar chart in 2012 also explains why november had such a spike in the stacked bar chart,
Next to this it also shows that 2011-2013 had significantly more crimes than the years after.

In [70]:
fig = px.bar(grouped2, x='Year', y='Count', color='Month')
fig.update_layout(xaxis_title="Year", yaxis_title="Crime Count")
# show the plot
fig.show()

In [71]:
fig = px.line(grouped, x='Month', y='Count')
fig.update_layout(xaxis_title="Year", yaxis_title="Count")
fig.add_vrect(x0='2020-03', x1='2020-04', fillcolor='red', opacity=0.5, layer='below', line_width=0) #Lockdown
fig.add_vrect(x0='2020-04', x1='2020-06', fillcolor='purple', opacity=0.5, layer='below', line_width=0) #extension of lockdown
fig.add_vrect(x0='2020-10', x1='2020-11', fillcolor='orange', opacity=0.4, layer='below', line_width=0) #Restrictions
fig.add_vrect(x0='2020-11', x1='2020-12', fillcolor='red', opacity=0.5, layer='below', line_width=0) #Lockdown
fig.add_vrect(x0='2020-12', x1='2021-01', fillcolor='orange', opacity=0.4, layer='below', line_width=0) #Restrictions
fig.add_vrect(x0='2021-01', x1='2021-02', fillcolor='red', opacity=0.5, layer='below', line_width=0) #Lockdown
fig.add_vrect(x0='2021-02', x1='2021-03', fillcolor='orange', opacity=0.4, layer='below', line_width=0) #Meetings allowed and schools opened
fig.add_vrect(x0='2021-03', x1='2021-07', fillcolor='yellow', opacity=0.3, layer='below', line_width=0) #slow release of restrictions.
fig.add_vrect(x0='2021-12', x1='2022-01', fillcolor='purple', opacity=0.5, layer='below', line_width=0) #extension of lockdown
#dates using: https://www.instituteforgovernment.org.uk/sites/default/files/2022-12/timeline-coronavirus-lockdown-december-2021.pdf

fig.show()

Here:
Red = Lockdown
Orange = restrictions
Yellow = slow release of restrictions
Purple = extension of current measures.

 # OVERALL FINDINGS.

It definitely seems that there are more crimes in the winter and fall months, so it would make sense to strengthen police force in these months, an option for this could be that more police have their holidays in this period of the year.

Considering the fact that november has a significantly higher amount of burglaries I started looking for reasons as to why this could be the case. There is a holiday in november 05-11 Guy Fawkes Day, with Guy Fawkes day including people going out to see bonfires in the night, which could be a good time for a burglar to strike. However, since our data only has the month of burglaries we can not make any conclusions about whether this is relevant in any way.
Another way to look at it is the fact that november has longer nights, which could also be an explanation for fall and winter months having higher burglary rates.

Using https://www.instituteforgovernment.org.uk/sites/default/files/2022-12/timeline-coronavirus-lockdown-december-2021.pdf We also studied the effect of corona on the burglary rate. It seems that during the lockdowns the crimes always decreased, the first 2 times especially. When an extension of the lockdown or measures was announced crime shot up again, but since this was only announced twice we can not make any conclusions from this.

